# Strategy Comparison: Credit-Greedy vs Forecast-Aware

This notebook compares the two carbon-aware scheduling strategies:

1. **Credit-Greedy**: Reactive approach using credit balance to trade off quality vs carbon
2. **Forecast-Aware**: Proactive approach using carbon intensity forecasts

## Key Questions
- Which strategy shows stronger carbon-aware behavior?
- How do they differ in p100 usage patterns?
- What are the tradeoffs between the two approaches?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import glob
import os

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Load Data from Both Strategies

In [ ]:
# Load credit-greedy data
cg_path = 'results/simple_20251113_204309/credit-greedy/timeseries.csv'
df_cg = pd.read_csv(cg_path)
df_cg['strategy'] = 'credit-greedy'

# Find most recent forecast-aware results
fa_dirs = glob.glob('results/simple_*/forecast-aware')
if not fa_dirs:
    raise FileNotFoundError("No forecast-aware results found. Run the benchmark first.")
fa_path = os.path.join(max(fa_dirs, key=os.path.getmtime), 'timeseries.csv')
df_fa = pd.read_csv(fa_path)
df_fa['strategy'] = 'forecast-aware'

# Calculate percentages for both
for df in [df_cg, df_fa]:
    df['total_requests'] = df['requests_precision_30'] + df['requests_precision_50'] + df['requests_precision_100']
    df['p30_pct'] = (df['requests_precision_30'] / df['total_requests']) * 100
    df['p50_pct'] = (df['requests_precision_50'] / df['total_requests']) * 100
    df['p100_pct'] = (df['requests_precision_100'] / df['total_requests']) * 100
    df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Credit-Greedy: {len(df_cg)} samples, {df_cg['total_requests'].sum():.0f} total requests")
print(f"Forecast-Aware: {len(df_fa)} samples, {df_fa['total_requests'].sum():.0f} total requests")

## 2. Carbon-Aware Behavior Comparison

In [ ]:
def analyze_carbon_awareness(df, name):
    """Analyze carbon-aware behavior for a strategy."""
    very_low = df[df['carbon_now'] <= 60]
    low = df[df['carbon_now'] <= 80]
    mid = df[(df['carbon_now'] > 80) & (df['carbon_now'] < 240)]
    high = df[df['carbon_now'] >= 240]
    very_high = df[df['carbon_now'] >= 280]
    
    print(f"\n{'=' * 80}")
    print(f"{name.upper()} CARBON-AWARE BEHAVIOR")
    print(f"{'=' * 80}")
    print(f"VERY LOW (≤60):   {len(very_low):3d} samples  |  p100: {very_low['p100_pct'].mean():5.1f}%  |  Weight: {very_low['commanded_weight_100'].mean():.1f}%")
    print(f"LOW (≤80):        {len(low):3d} samples  |  p100: {low['p100_pct'].mean():5.1f}%  |  Weight: {low['commanded_weight_100'].mean():.1f}%")
    print(f"MID (80-240):     {len(mid):3d} samples  |  p100: {mid['p100_pct'].mean():5.1f}%  |  Weight: {mid['commanded_weight_100'].mean():.1f}%")
    print(f"HIGH (≥240):      {len(high):3d} samples  |  p100: {high['p100_pct'].mean():5.1f}%  |  Weight: {high['commanded_weight_100'].mean():.1f}%")
    print(f"VERY HIGH (≥280): {len(very_high):3d} samples  |  p100: {very_high['p100_pct'].mean():5.1f}%  |  Weight: {very_high['commanded_weight_100'].mean():.1f}%")
    print()
    
    swing = low['p100_pct'].mean() - high['p100_pct'].mean()
    swing_vl_vh = very_low['p100_pct'].mean() - very_high['p100_pct'].mean()
    
    print(f"P100 SWING (Low - High): {swing:+.1f} percentage points")
    print(f"P100 SWING (Very Low - Very High): {swing_vl_vh:+.1f} percentage points")
    
    return {
        'swing': swing,
        'swing_extreme': swing_vl_vh,
        'low_p100': low['p100_pct'].mean(),
        'high_p100': high['p100_pct'].mean(),
        'verylow_p100': very_low['p100_pct'].mean(),
        'veryhigh_p100': very_high['p100_pct'].mean()
    }

cg_stats = analyze_carbon_awareness(df_cg, 'Credit-Greedy')
fa_stats = analyze_carbon_awareness(df_fa, 'Forecast-Aware')

print(f"\n{'=' * 80}")
print("COMPARISON SUMMARY")
print(f"{'=' * 80}")
print(f"Credit-Greedy swing:    {cg_stats['swing']:+.1f} points")
print(f"Forecast-Aware swing:   {fa_stats['swing']:+.1f} points")
print(f"Difference:             {fa_stats['swing'] - cg_stats['swing']:+.1f} points")
print()
if abs(fa_stats['swing']) > abs(cg_stats['swing']):
    print("✅ Forecast-Aware shows STRONGER carbon-aware behavior")
elif abs(cg_stats['swing']) > abs(fa_stats['swing']):
    print("✅ Credit-Greedy shows STRONGER carbon-aware behavior")
else:
    print("⚖️ Both strategies show similar carbon-aware behavior")

## 3. Side-by-Side Precision Distribution

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Credit-Greedy
axes[0].fill_between(df_cg['elapsed_seconds'], 0, df_cg['p30_pct'], label='p30', alpha=0.7, color='#e74c3c')
axes[0].fill_between(df_cg['elapsed_seconds'], df_cg['p30_pct'], df_cg['p30_pct'] + df_cg['p50_pct'], label='p50', alpha=0.7, color='#f39c12')
axes[0].fill_between(df_cg['elapsed_seconds'], df_cg['p30_pct'] + df_cg['p50_pct'], 100, label='p100', alpha=0.7, color='#2ecc71')
axes[0].set_ylabel('Percentage of Requests (%)')
axes[0].set_title('Credit-Greedy: Request Distribution by Precision Level')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, 100)

# Forecast-Aware
axes[1].fill_between(df_fa['elapsed_seconds'], 0, df_fa['p30_pct'], label='p30', alpha=0.7, color='#e74c3c')
axes[1].fill_between(df_fa['elapsed_seconds'], df_fa['p30_pct'], df_fa['p30_pct'] + df_fa['p50_pct'], label='p50', alpha=0.7, color='#f39c12')
axes[1].fill_between(df_fa['elapsed_seconds'], df_fa['p30_pct'] + df_fa['p50_pct'], 100, label='p100', alpha=0.7, color='#2ecc71')
axes[1].set_xlabel('Elapsed Time (seconds)')
axes[1].set_ylabel('Percentage of Requests (%)')
axes[1].set_title('Forecast-Aware: Request Distribution by Precision Level')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 100)

plt.tight_layout()
plt.show()

print("\nMean Precision Distribution:")
print(f"{'Strategy':<20} {'p30':<8} {'p50':<8} {'p100':<8}")
print(f"{'-'*44}")
print(f"{'Credit-Greedy':<20} {df_cg['p30_pct'].mean():6.1f}%  {df_cg['p50_pct'].mean():6.1f}%  {df_cg['p100_pct'].mean():6.1f}%")
print(f"{'Forecast-Aware':<20} {df_fa['p30_pct'].mean():6.1f}%  {df_fa['p50_pct'].mean():6.1f}%  {df_fa['p100_pct'].mean():6.1f}%")
print(f"{'-'*44}")
print(f"{'Difference':<20} {df_fa['p30_pct'].mean() - df_cg['p30_pct'].mean():+6.1f}%  {df_fa['p50_pct'].mean() - df_cg['p50_pct'].mean():+6.1f}%  {df_fa['p100_pct'].mean() - df_cg['p100_pct'].mean():+6.1f}%")

## 4. p100 Usage Comparison: Low vs High Carbon

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Box plot comparison
carbon_categories_cg = pd.cut(df_cg['carbon_now'], bins=[0, 80, 160, 240, 400], 
                               labels=['Very Low', 'Low', 'Medium', 'High'])
carbon_categories_fa = pd.cut(df_fa['carbon_now'], bins=[0, 80, 160, 240, 400], 
                               labels=['Very Low', 'Low', 'Medium', 'High'])

df_cg['carbon_category'] = carbon_categories_cg
df_fa['carbon_category'] = carbon_categories_fa

# Combine for side-by-side boxplot
df_combined = pd.concat([df_cg[['carbon_category', 'p100_pct', 'strategy']], 
                          df_fa[['carbon_category', 'p100_pct', 'strategy']]])

import seaborn as sns
sns.boxplot(data=df_combined, x='carbon_category', y='p100_pct', hue='strategy', ax=axes[0])
axes[0].set_xlabel('Carbon Level')
axes[0].set_ylabel('p100 Usage (%)')
axes[0].set_title('p100 Usage by Carbon Level: Strategy Comparison')
axes[0].legend(title='Strategy')
axes[0].grid(True, alpha=0.3, axis='y')

# Bar chart of mean values
categories = ['Very Low\n(≤80)', 'Low\n(80-160)', 'Mid\n(160-240)', 'High\n(≥240)']
cg_means = [df_cg[df_cg['carbon_now'] <= 80]['p100_pct'].mean(),
            df_cg[(df_cg['carbon_now'] > 80) & (df_cg['carbon_now'] <= 160)]['p100_pct'].mean(),
            df_cg[(df_cg['carbon_now'] > 160) & (df_cg['carbon_now'] <= 240)]['p100_pct'].mean(),
            df_cg[df_cg['carbon_now'] > 240]['p100_pct'].mean()]
fa_means = [df_fa[df_fa['carbon_now'] <= 80]['p100_pct'].mean(),
            df_fa[(df_fa['carbon_now'] > 80) & (df_fa['carbon_now'] <= 160)]['p100_pct'].mean(),
            df_fa[(df_fa['carbon_now'] > 160) & (df_fa['carbon_now'] <= 240)]['p100_pct'].mean(),
            df_fa[df_fa['carbon_now'] > 240]['p100_pct'].mean()]

x = np.arange(len(categories))
width = 0.35
axes[1].bar(x - width/2, cg_means, width, label='Credit-Greedy', alpha=0.8, color='#3498db')
axes[1].bar(x + width/2, fa_means, width, label='Forecast-Aware', alpha=0.8, color='#e74c3c')
axes[1].set_xlabel('Carbon Level')
axes[1].set_ylabel('Mean p100 Usage (%)')
axes[1].set_title('Mean p100 Usage by Carbon Level')
axes[1].set_xticks(x)
axes[1].set_xticklabels(categories)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Scatter Plot: p100 Usage vs Carbon Intensity

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Credit-Greedy
axes[0].scatter(df_cg['carbon_now'], df_cg['p100_pct'], alpha=0.5, s=30, color='#3498db')
z_cg = np.polyfit(df_cg['carbon_now'], df_cg['p100_pct'], 1)
p_cg = np.poly1d(z_cg)
axes[0].plot(df_cg['carbon_now'].sort_values(), p_cg(df_cg['carbon_now'].sort_values()), 
             "r--", alpha=0.8, linewidth=2, label=f'Trend: {z_cg[0]:.4f}x + {z_cg[1]:.2f}')
axes[0].set_xlabel('Carbon Intensity (gCO2/kWh)')
axes[0].set_ylabel('p100 Usage (%)')
axes[0].set_title('Credit-Greedy: p100 vs Carbon')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Forecast-Aware
axes[1].scatter(df_fa['carbon_now'], df_fa['p100_pct'], alpha=0.5, s=30, color='#e74c3c')
z_fa = np.polyfit(df_fa['carbon_now'], df_fa['p100_pct'], 1)
p_fa = np.poly1d(z_fa)
axes[1].plot(df_fa['carbon_now'].sort_values(), p_fa(df_fa['carbon_now'].sort_values()), 
             "r--", alpha=0.8, linewidth=2, label=f'Trend: {z_fa[0]:.4f}x + {z_fa[1]:.2f}')
axes[1].set_xlabel('Carbon Intensity (gCO2/kWh)')
axes[1].set_ylabel('p100 Usage (%)')
axes[1].set_title('Forecast-Aware: p100 vs Carbon')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTrend Line Slopes:")
print(f"  Credit-Greedy:   {z_cg[0]:.6f} (negative = p100 decreases as carbon increases)")
print(f"  Forecast-Aware:  {z_fa[0]:.6f}")
print()
if z_cg[0] < z_fa[0]:
    print(f"Credit-Greedy has a steeper negative slope → stronger carbon response")
elif z_fa[0] < z_cg[0]:
    print(f"Forecast-Aware has a steeper negative slope → stronger carbon response")
else:
    print(f"Both strategies have similar slopes")

## 6. Quality vs Carbon Tradeoff

Calculate the mean precision (quality) and weighted carbon cost for each strategy.

In [ ]:
# Carbon cost per precision level (from carbonstat values.yaml)
carbon_intensity = {'p30': 0.3, 'p50': 0.6, 'p100': 1.0}

def calculate_metrics(df, name):
    """Calculate quality and carbon metrics."""
    # Mean precision (quality proxy)
    mean_quality = df['mean_precision'].mean()
    
    # Weighted carbon cost
    weighted_carbon = (
        df['p30_pct'].mean() * carbon_intensity['p30'] +
        df['p50_pct'].mean() * carbon_intensity['p50'] +
        df['p100_pct'].mean() * carbon_intensity['p100']
    ) / 100
    
    # Carbon responsiveness (standard deviation of p100 usage)
    p100_std = df['p100_pct'].std()
    
    print(f"\n{name}:")
    print(f"  Mean Quality (precision): {mean_quality:.4f}")
    print(f"  Weighted Carbon Cost:     {weighted_carbon:.4f}")
    print(f"  p100 Std Dev:             {p100_std:.2f}%")
    print(f"  Total Requests:           {df['total_requests'].sum():.0f}")
    
    return {
        'quality': mean_quality,
        'carbon': weighted_carbon,
        'p100_std': p100_std,
        'total_requests': df['total_requests'].sum()
    }

cg_metrics = calculate_metrics(df_cg, 'Credit-Greedy')
fa_metrics = calculate_metrics(df_fa, 'Forecast-Aware')

print(f"\n{'=' * 80}")
print("TRADEOFF ANALYSIS")
print(f"{'=' * 80}")
print(f"Quality Difference:  {fa_metrics['quality'] - cg_metrics['quality']:+.4f} (FA - CG)")
print(f"Carbon Difference:   {fa_metrics['carbon'] - cg_metrics['carbon']:+.4f} (FA - CG)")
print(f"p100 Variability:    {fa_metrics['p100_std'] - cg_metrics['p100_std']:+.2f}% (FA - CG)")
print()

# Visualization
fig, ax = plt.subplots(figsize=(8, 6))
strategies = ['Credit-Greedy', 'Forecast-Aware']
quality_values = [cg_metrics['quality'], fa_metrics['quality']]
carbon_values = [cg_metrics['carbon'], fa_metrics['carbon']]

ax.scatter(carbon_values, quality_values, s=500, alpha=0.6, c=['#3498db', '#e74c3c'])
for i, strategy in enumerate(strategies):
    ax.annotate(strategy, (carbon_values[i], quality_values[i]), 
                xytext=(10, 10), textcoords='offset points', fontsize=12)

ax.set_xlabel('Weighted Carbon Cost')
ax.set_ylabel('Mean Quality (Precision)')
ax.set_title('Quality vs Carbon Cost Tradeoff')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Summary and Recommendations

In [ ]:
print("=" * 80)
print("STRATEGY COMPARISON SUMMARY")
print("=" * 80)
print()
print("CARBON-AWARE BEHAVIOR:")
print(f"  Credit-Greedy p100 swing:    {cg_stats['swing']:+.1f} points")
print(f"  Forecast-Aware p100 swing:   {fa_stats['swing']:+.1f} points")
winner = "Forecast-Aware" if abs(fa_stats['swing']) > abs(cg_stats['swing']) else "Credit-Greedy"
print(f"  Winner: {winner}")
print()
print("QUALITY:")
print(f"  Credit-Greedy mean quality:  {cg_metrics['quality']:.4f}")
print(f"  Forecast-Aware mean quality: {fa_metrics['quality']:.4f}")
print(f"  Difference: {fa_metrics['quality'] - cg_metrics['quality']:+.4f}")
print()
print("CARBON COST:")
print(f"  Credit-Greedy carbon cost:   {cg_metrics['carbon']:.4f}")
print(f"  Forecast-Aware carbon cost:  {fa_metrics['carbon']:.4f}")
print(f"  Difference: {fa_metrics['carbon'] - cg_metrics['carbon']:+.4f}")
print()
print("RESPONSIVENESS (p100 variability):")
print(f"  Credit-Greedy:   {cg_metrics['p100_std']:.2f}% std dev")
print(f"  Forecast-Aware:  {fa_metrics['p100_std']:.2f}% std dev")
print()
print("=" * 80)
print("KEY INSIGHTS:")
print("=" * 80)

if abs(fa_stats['swing']) > abs(cg_stats['swing']) + 5:
    print("✅ Forecast-Aware shows SIGNIFICANTLY stronger carbon-aware behavior")
elif abs(cg_stats['swing']) > abs(fa_stats['swing']) + 5:
    print("✅ Credit-Greedy shows SIGNIFICANTLY stronger carbon-aware behavior")
else:
    print("⚖️ Both strategies show similar carbon-aware behavior")

if cg_metrics['quality'] > fa_metrics['quality']:
    print(f"📊 Credit-Greedy provides {((cg_metrics['quality']/fa_metrics['quality'] - 1) * 100):.1f}% higher quality")
else:
    print(f"📊 Forecast-Aware provides {((fa_metrics['quality']/cg_metrics['quality'] - 1) * 100):.1f}% higher quality")

if cg_metrics['carbon'] < fa_metrics['carbon']:
    print(f"🌱 Credit-Greedy has {((1 - cg_metrics['carbon']/fa_metrics['carbon']) * 100):.1f}% lower carbon cost")
else:
    print(f"🌱 Forecast-Aware has {((1 - fa_metrics['carbon']/cg_metrics['carbon']) * 100):.1f}% lower carbon cost")

print()
print("RECOMMENDATION:")
if abs(fa_stats['swing']) > 15:
    print("  → Forecast-Aware is highly responsive to carbon changes")
elif abs(cg_stats['swing']) > 15:
    print("  → Credit-Greedy is highly responsive to carbon changes")
else:
    print("  → Both strategies show weak carbon-aware behavior (<15 point swing)")
    print("  → Consider tuning algorithm parameters for stronger carbon response")